In [15]:
# library importation 
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
st_words = stopwords.words('english')
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string
import numpy as np
import pandas as pd
import seaborn as sn
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [16]:
# loading BBC news dataset.... 
data = pd.read_csv('bbc_text.csv')
data.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [17]:
lemmer = WordNetLemmatizer()


def dataset_cleaning(text):
    #  removeing punctuation from the dataset
    text = ''.join([token for token in text if not token in string.punctuation])
    token = word_tokenize(text)
    # removing stopword (words that had less meaning to the dataset)     
    text  = [t.lower() for t in token if not t in st_words ]
    text_lemma = [lemmer.lemmatize(t) for t in text]
    
    return text_lemma

data['lem_clean_text'] = data["text"].apply(lambda x: dataset_cleaning(x))

clean_data = data.drop(columns=['text'])
clean_data.head()

,category,lem_clean_text
0,tech,"[tv, future, hand, viewer, home, theatre, syst..."
1,business,"[worldcom, bos, left, book, alone, former, wor..."
2,sport,"[tiger, wary, farrell, gamble, leicester, say,..."
3,sport,"[yeading, face, newcastle, fa, cup, premiershi..."
4,entertainment,"[ocean, twelve, raid, box, office, ocean, twel..."


In [18]:
# creating a document version of the lemmatization

def return_doc(documents): 
    return ' '.join(documents)

clean_data['doc'] = clean_data['lem_clean_text'].apply(lambda x: return_doc(x))
clean_data.head(3)

,category,lem_clean_text,doc
0,tech,"[tv, future, hand, viewer, home, theatre, syst...",tv future hand viewer home theatre system plas...
1,business,"[worldcom, bos, left, book, alone, former, wor...",worldcom bos left book alone former worldcom b...
2,sport,"[tiger, wary, farrell, gamble, leicester, say,...",tiger wary farrell gamble leicester say rushed...


In [19]:
clean_data.category = clean_data.category.astype('category')
encoded_lbl = clean_data.category.cat.codes
clean_data.category.unique()

clean_data['lbl'] = encoded_lbl
clean_data[['category', 'lbl']]

label_encode =  dict(zip( clean_data.category.unique() , encoded_lbl.unique() ))
label_encode

y = clean_data.lbl
print(y[:10])

0    4
1    0
2    3
3    3
4    1
5    2
6    2
7    3
8    3
9    1
Name: lbl, dtype: int8


In [20]:
docs = [
    "apple and banana are healthy",               # Class 1
    "eat fresh fruits every day",                 # Class 1
    "machine learning is fascinating",            # Class 2
    "deep learning uses neural networks",         # Class 2
    "visit the beach this summer",                # Class 3
    "swimming is fun in the ocean",               # Class 3
    "python and java are programming languages",  # Class 4
    "code in c++ or python for AI",               # Class 4
    "mango and apple for breakfast",              # Class 1
    "neural networks are part of AI"              # Class 2
]

In [21]:
clean_data.doc

0       tv future hand viewer home theatre system plas...
1       worldcom bos left book alone former worldcom b...
2       tiger wary farrell gamble leicester say rushed...
3       yeading face newcastle fa cup premiership side...
4       ocean twelve raid box office ocean twelve crim...
                              ...                        
2220    car pull u retail figure u retail sale fell 03...
2221    kilroy unveils immigration policy exchatshow h...
2222    rem announce new glasgow concert u band rem an...
2223    political squabble snowball become commonplace...
2224    souness delight euro progress bos graeme soune...
Name: doc, Length: 2225, dtype: object

In [22]:
import math

def tf_idf_preprocess(doc):
    return [word for word in doc.lower().split() if not any(char.isdigit() for char in word)]
    return doc

clean_doc = [tf_idf_preprocess(doc) for doc in clean_data.doc]
# clean_doc = [tf_idf_preprocess(doc) for doc in docs]
# clean_doc

In [24]:
# print(clean_doc)

In [25]:
def get_vocabulary(documents):
    vocab_list = []
    [[vocab_list.append(d) for d in doc] for doc in documents]
    
    return sorted(set(vocab_list))

vocabulary  = get_vocabulary(clean_doc)

In [26]:
def compute_tf(doc, vocab):
    tf = {}
    total_terms = len(doc)
    for term in vocab:
        tf[term] = doc.count(term) / total_terms
    return tf

tf_list = np.array([compute_tf(doc, vocabulary) for doc in clean_doc])
# tf_list

In [ ]:
df = {}
for term in vocabulary:
    df[term] = sum(1 for doc in clean_doc if term in doc)

In [20]:
# N = len(clean_doc)
# idf = {}
# for term in vocabulary:
#     idf[term] = math.log(N / df[term])


def get_idf(clean_doc, vocabulary): 
    N = len(clean_doc)
    idf = {}
    for term in vocabulary:
        idf[term] = math.log(N / df[term])
    return idf

idf = get_idf(clean_doc, vocabulary)
idf

{'aa': 7.707512194600341,
 'aaa': 4.999461993498131,
 'aac': 7.014365014040395,
 'aadc': 7.707512194600341,
 'aaliyah': 7.707512194600341,
 'aaltra': 7.707512194600341,
 'aamir': 7.707512194600341,
 'aan': 7.707512194600341,
 'aara': 7.707512194600341,
 'aarhus': 7.707512194600341,
 'aaron': 6.09807428216624,
 'abacus': 7.014365014040395,
 'abandon': 6.608899905932231,
 'abandoned': 4.8742988505441245,
 'abandoning': 5.761602045545027,
 'abandonment': 6.09807428216624,
 'abate': 7.707512194600341,
 'abatement': 7.707512194600341,
 'abating': 7.014365014040395,
 'abba': 6.32121783348045,
 'abbas': 6.32121783348045,
 'abbasi': 6.32121783348045,
 'abbey': 7.014365014040395,
 'abbot': 7.707512194600341,
 'abbott': 5.5102876172641215,
 'abbreviated': 7.707512194600341,
 'abc': 5.30961692180197,
 'abd': 7.707512194600341,
 'abdellatif': 7.014365014040395,
 'abdication': 7.014365014040395,
 'abdomen': 7.707512194600341,
 'abdominal': 7.014365014040395,
 'abducted': 7.707512194600341,
 'abduct

In [22]:
tfidf_list = []
for tf in tf_list:
    tfidf = {term: tf[term] * idf[term] for term in vocabulary}
    tfidf_list.append(tfidf)

In [ ]:
# tfidf_list
[12,4,5,6] + [0] * 5

[12, 4, 5, 6, 0, 0, 0, 0, 0]

In [24]:
# clean_doc
clean_data['lem_clean_text'].apply(lambda x : len(x)).mean()

222.02966292134832

In [25]:
# [docs for docs in clean_doc]
def tf_idf_embedding(clean_doc, tfidf_list, padding):
    vectors = []
    for index in range(len(clean_doc)):
        doc = clean_doc[index]
        vecs = tfidf_list[index]
        doc_vec = [vecs[d] for d in doc]
        doc_vec = doc_vec + [0] * (padding-len(doc_vec))  if len(doc_vec) < padding else doc_vec[:padding]
        vectors.append(doc_vec)
    return np.array(vectors)

vectors  = tf_idf_embedding(clean_doc, tfidf_list, 50)
vectors

array([[0.08263126, 0.01029649, 0.01309365, ..., 0.00960117, 0.03577394,
        0.00765443],
       [0.21975657, 0.04706838, 0.01326488, ..., 0.04066131, 0.06011653,
        0.0335059 ],
       [0.0738461 , 0.04271541, 0.21814227, ..., 0.01948902, 0.02509252,
        0.01948902],
       ...,
       [0.18479013, 0.0242057 , 0.00996367, ..., 0.14130272, 0.01626443,
        0.0349188 ],
       [0.00683008, 0.0161668 , 0.01971231, ..., 0.04318213, 0.01278635,
        0.01533424],
       [0.18309024, 0.04750248, 0.02569073, ..., 0.09900794, 0.03159018,
        0.01063467]])

In [162]:
# BestNeighbourTF-IDF

In [26]:
import numpy as np

def best_cosine_similarity(vectors):
    # Normalize rows
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    normalized_vectors = vectors / norms

    # Compute cosine similarity matrix
    cosine_sim_matrix = normalized_vectors @ normalized_vectors.T

    # Remove self-similarity by setting diagonal to -1
    np.fill_diagonal(cosine_sim_matrix, -1)

    # Get the maximum similarity (best match) for each row
    best_similarities = np.max(cosine_sim_matrix, axis=1)

    return best_similarities

cosine = best_cosine_similarity(vectors)
cosine

array([0.85458067, 0.82115838, 0.87947253, ..., 0.82858481, 0.81332119,
       0.87368868])

In [27]:
X = pd.DataFrame(vectors)
X['NBTF-IDF'] =  cosine
X

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,NBTF-IDF
0,0.082631,0.010296,0.013094,0.035774,0.008361,0.009548,0.010705,0.014261,0.043320,0.082631,...,0.016359,0.017044,0.006882,0.017567,0.022299,0.030520,0.009601,0.035774,0.007654,0.854581
1,0.219757,0.047068,0.013265,0.032366,0.019413,0.031210,0.219757,0.047068,0.028696,0.204216,...,0.017227,0.011838,0.067774,0.167530,0.009798,0.024663,0.040661,0.060117,0.033506,0.821158
2,0.073846,0.042715,0.218142,0.128146,0.091775,0.009189,0.039290,0.016465,0.021780,0.049391,...,0.049391,0.218142,0.039511,0.032187,0.022966,0.043628,0.019489,0.025093,0.019489,0.879473
3,0.117225,0.031633,0.042207,0.030189,0.041119,0.038978,0.041713,0.042207,0.039660,0.031633,...,0.019009,0.031881,0.015576,0.012414,0.039660,0.025642,0.028138,0.043525,0.006422,0.839863
4,0.153590,0.096538,0.033396,0.056046,0.040641,0.153590,0.096538,0.019508,0.042511,0.130494,...,0.014724,0.013249,0.026099,0.005432,0.008329,0.017007,0.009195,0.013129,0.034919,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2220,0.115912,0.023779,0.029103,0.137711,0.037114,0.029103,0.137711,0.171015,0.017686,0.053205,...,0.053205,0.021282,0.012635,0.027316,0.029103,0.137711,0.048846,0.031326,0.041075,0.786119
2221,0.062271,0.051947,0.075719,0.052821,0.073835,0.032345,0.031463,0.162397,0.043122,0.015221,...,0.064190,0.042301,0.041298,0.038573,0.023662,0.067766,0.184223,0.021522,0.010960,0.808345
2222,0.184790,0.024206,0.009964,0.184130,0.149963,0.005962,0.038484,0.184790,0.028727,0.010663,...,0.030412,0.032038,0.048119,0.080108,0.149963,0.076708,0.141303,0.016264,0.034919,0.828585
2223,0.006830,0.016167,0.019712,0.005581,0.014093,0.010770,0.019293,0.022030,0.003598,0.013823,...,0.021463,0.008636,0.014936,0.015334,0.011461,0.033644,0.043182,0.012786,0.015334,0.813321


In [22]:
# tfidf_X_train, tfidf_X_test, tfidf_y_train, tfidf_y_test = train_test_split(X, y , test_size=0.2, stratify=y, random_state=42)

In [199]:
# tfidf_y_train

In [26]:
# clean data... 
clean_data.head()

,category,lem_clean_text,doc,lbl
0,tech,"[tv, future, hand, viewer, home, theatre, syst...",tv future hand viewer home theatre system plas...,4
1,business,"[worldcom, bos, left, book, alone, former, wor...",worldcom bos left book alone former worldcom b...,0
2,sport,"[tiger, wary, farrell, gamble, leicester, say,...",tiger wary farrell gamble leicester say rushed...,3
3,sport,"[yeading, face, newcastle, fa, cup, premiershi...",yeading face newcastle fa cup premiership side...,3
4,entertainment,"[ocean, twelve, raid, box, office, ocean, twel...",ocean twelve raid box office ocean twelve crim...,1


In [27]:
y.sample(5)

2170    4
1910    2
135     4
2186    0
1742    4
Name: lbl, dtype: int8

In [28]:
doc1 =  clean_data['lem_clean_text']
doc2 = clean_data['doc']
doc2

0       tv future hand viewer home theatre system plas...
1       worldcom bos left book alone former worldcom b...
2       tiger wary farrell gamble leicester say rushed...
3       yeading face newcastle fa cup premiership side...
4       ocean twelve raid box office ocean twelve crim...
                              ...                        
2220    car pull u retail figure u retail sale fell 03...
2221    kilroy unveils immigration policy exchatshow h...
2222    rem announce new glasgow concert u band rem an...
2223    political squabble snowball become commonplace...
2224    souness delight euro progress bos graeme soune...
Name: doc, Length: 2225, dtype: object

In [29]:
tf = TfidfVectorizer()
tf_vec = tf.fit_transform(doc2)
tf_vec

<2225x29599 sparse matrix of type '<class 'numpy.float64'>'
	with 339004 stored elements in Compressed Sparse Row format>

In [46]:
tf_vec.toarray()

array([[0.        , 0.01930694, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.02380416, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.02588097, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.0160233 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [48]:
X = tf_vec.toarray()
X.shape

(2225, 29599)

In [49]:
tfidf_X_train, tfidf_X_test, tfidf_y_train, tfidf_y_test = train_test_split(X, y , test_size=0.2, stratify=y, random_state=42)

In [53]:
# LSTM TFIDF... 

# TF-IDF WITH WORD2VEC....

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define input shape
# input_shape = (X_train.shape[1],)  # Number of features

# Build the ANN model
lstm_tfidf_model = Sequential([
    Dense(64, activation='relu', input_shape=(29599,)),
    Dense(32, activation='relu'),
    keras.layers.Dropout(0.5),
    Dense(16, activation='relu'),
    Dense(5, activation='sigmoid')  # 5-class output
])

# Compile the model
lstm_tfidf_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Summary of the model
lstm_tfidf_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                1894400   
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_14 (Dense)            (None, 16)                528       
                                                                 
 dense_15 (Dense)            (None, 5)                 85        
                                                                 
Total params: 1,897,093
Trainable params: 1,897,093
Non-trainable params: 0
_________________________________________________________________


In [54]:
lstm_tfidf_model.fit(tfidf_X_train, tfidf_y_train,validation_data=(tfidf_X_test, tfidf_y_test) , epochs=10)

Epoch 1/10
56/56 [==============================] - 4s 41ms/step - loss: 1.4689 - accuracy: 0.5157 - val_loss: 1.1097 - val_accuracy: 0.8629
Epoch 2/10
56/56 [==============================] - 2s 31ms/step - loss: 0.7657 - accuracy: 0.8287 - val_loss: 0.3408 - val_accuracy: 0.9798
Epoch 3/10
56/56 [==============================] - 2s 33ms/step - loss: 0.2638 - accuracy: 0.9590 - val_loss: 0.1255 - val_accuracy: 0.9843
Epoch 4/10
56/56 [==============================] - 2s 33ms/step - loss: 0.1239 - accuracy: 0.9753 - val_loss: 0.0866 - val_accuracy: 0.9775
Epoch 5/10
56/56 [==============================] - 2s 31ms/step - loss: 0.0710 - accuracy: 0.9865 - val_loss: 0.0742 - val_accuracy: 0.9753
Epoch 6/10
56/56 [==============================] - 2s 33ms/step - loss: 0.0592 - accuracy: 0.9876 - val_loss: 0.0742 - val_accuracy: 0.9775
Epoch 7/10
56/56 [==============================] - 2s 31ms/step - loss: 0.0426 - accuracy: 0.9921 - val_loss: 0.0687 - val_accuracy: 0.9775
Epoch 8/10
56

In [56]:
doc1

0       [tv, future, hand, viewer, home, theatre, syst...
1       [worldcom, bos, left, book, alone, former, wor...
2       [tiger, wary, farrell, gamble, leicester, say,...
3       [yeading, face, newcastle, fa, cup, premiershi...
4       [ocean, twelve, raid, box, office, ocean, twel...
                              ...                        
2220    [car, pull, u, retail, figure, u, retail, sale...
2221    [kilroy, unveils, immigration, policy, exchats...
2222    [rem, announce, new, glasgow, concert, u, band...
2223    [political, squabble, snowball, become, common...
2224    [souness, delight, euro, progress, bos, graeme...
Name: lem_clean_text, Length: 2225, dtype: object

In [30]:
tfid_ngrams = TfidfVectorizer(sublinear_tf=True)
tfidngram_vec = tfid_ngrams.fit_transform(doc2)

In [46]:
# reduce dimension... 
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=500)
reduced_tfidf = svd.fit_transform(tfidngram_vec.toarray())

In [47]:
reduced_tfidf

array([[ 0.25482679, -0.01517531, -0.16272868, ..., -0.00251347,
        -0.0193776 , -0.00680958],
       [ 0.1409452 ,  0.02134469, -0.00640912, ..., -0.01963276,
         0.00438885,  0.02136215],
       [ 0.18919921, -0.09691733,  0.07796327, ...,  0.01722919,
         0.00244093, -0.02649475],
       ...,
       [ 0.13242436, -0.05640021, -0.01512322, ...,  0.00616819,
        -0.05675641,  0.01237537],
       [ 0.19543432,  0.03071274,  0.05224818, ..., -0.00863946,
        -0.02095174,  0.04473564],
       [ 0.17994467, -0.14369712,  0.09271764, ..., -0.01420113,
        -0.01362199, -0.01038922]])

In [48]:
tfidf_X_train, tfidf_X_test, tfidf_y_train, tfidf_y_test = train_test_split(reduced_tfidf, y , test_size=0.2, stratify=y, random_state=42)

In [49]:
tfidf_X_train.shape

(1780, 500)

In [51]:
# LSTM TFIDF... 

# TF-IDF WITH WORD2VEC....
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define input shape
# input_shape = (X_train.shape[1],)  # Number of features

# Build the ANN model
lstm_tfidf_model = Sequential([
    Dense(64, activation='relu', input_shape=(500,)),
    Dense(32, activation='relu'),
    keras.layers.Dropout(0.5),
    Dense(16, activation='relu'),
    Dense(5, activation='sigmoid')  # 5-class output
])

# Compile the model
lstm_tfidf_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Summary of the model
lstm_tfidf_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                32064     
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_14 (Dense)            (None, 16)                528       
                                                                 
 dense_15 (Dense)            (None, 5)                 85        
                                                                 
Total params: 34,757
Trainable params: 34,757
Non-trainable params: 0
_________________________________________________________________


In [52]:
lstm_tfidf_model.fit(tfidf_X_train, tfidf_y_train,validation_data=(tfidf_X_test, tfidf_y_test) , epochs=10)

Epoch 1/10
56/56 [==============================] - 1s 7ms/step - loss: 1.5897 - accuracy: 0.2607 - val_loss: 1.5540 - val_accuracy: 0.3618
Epoch 2/10
56/56 [==============================] - 0s 4ms/step - loss: 1.4841 - accuracy: 0.3848 - val_loss: 1.3511 - val_accuracy: 0.5326
Epoch 3/10
56/56 [==============================] - 0s 6ms/step - loss: 1.1461 - accuracy: 0.6017 - val_loss: 0.8249 - val_accuracy: 0.7933
Epoch 4/10
56/56 [==============================] - 0s 5ms/step - loss: 0.6568 - accuracy: 0.8466 - val_loss: 0.3341 - val_accuracy: 0.9685
Epoch 5/10
56/56 [==============================] - 0s 4ms/step - loss: 0.3245 - accuracy: 0.9416 - val_loss: 0.1576 - val_accuracy: 0.9798
Epoch 6/10
56/56 [==============================] - 0s 4ms/step - loss: 0.1878 - accuracy: 0.9607 - val_loss: 0.0933 - val_accuracy: 0.9798
Epoch 7/10
56/56 [==============================] - 0s 4ms/step - loss: 0.1256 - accuracy: 0.9742 - val_loss: 0.0649 - val_accuracy: 0.9843
Epoch 8/10
56/56 [==